In [1]:
%%time

# Import the Python library that uses ODBC connections and Pandas
import pandas as pd
import numpy as np
import pyodbc
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

# Create a connection to Teradata
conn = pyodbc.connect('DSN=TDPROD') 

# Write the query you want to run
query_string = """SELECT ACF.CASE_NBR,ACF.CASE_CREATED_DATETM,ACF.CASE_CLOSED_DATETM,ACF.SUBJECT,AD.LODG_PROPERTY_NAME,AD.EXPE_LODG_PROPERTY_ID,AD.PROPERTY_SUPER_REGN_NAME,ACF.CASE_DESC,RTD.REC_TYP_NAME,CTD.CASE_TYP_NAME FROM P_DM_GSO_CASE.OPS_AGNT_CASE_FACT ACF JOIN P_DM_GSO_CASE.CASE_TYP_DIM CTD on CTD.CASE_TYP_KEY = ACF.CASE_TYP_KEY JOIN P_DM_GSO_CASE.REC_TYP_DIM RTD on RTD.REC_TYP_KEY = ACF.REC_TYP_KEY JOIN P_DM_GSO_CASE.GSO_ACCT_DIM AD on AD.ACCT_KEY = ACF.ACCT_KEY WHERE ACF.CASE_CREATED_DATETM > '2022-01-01 00:00:00' AND ACF.CASE_CREATED_EMP_KEY = '2536788' AND RTD.REC_TYP_NAME = 'Property Setup' and CTD.CASE_TYP_NAME = 'Pre-Onboarding' ORDER BY ACF.CASE_NBR;""" 

# Execute the query and store it in a Python data frame
df = pd.read_sql(query_string, conn)

# Close the connection
conn.close()

# Strip the time from created and closed datetimes
df["CASE_CREATED_DATE"] = df.CASE_CREATED_DATETM.dt.date
df["CASE_CLOSED_DATE"] = df.CASE_CLOSED_DATETM.dt.date
df.CASE_CREATED_DATE = pd.to_datetime(df.CASE_CREATED_DATE)
df.CASE_CLOSED_DATE = pd.to_datetime(df.CASE_CLOSED_DATE)

#save dataframe to csv file with semicolon separator
df.to_csv('C:/Users/clistewart/Downloads/copycatchercases.csv', sep=';') 

# Add all days and merge with data
from datetime import date, timedelta
today = date.today()
all_dates = pd.DataFrame(pd.date_range("2022-01-19", date.today() - timedelta(days = 1 )), columns=['CASE_CREATED_DATE'])
all_dates['PROPERTY_SUPER_REGN_NAME']='AMER'
all_dates2 = pd.DataFrame(pd.date_range("2022-01-19", date.today() - timedelta(days = 1 )), columns=['CASE_CREATED_DATE'])
all_dates2['PROPERTY_SUPER_REGN_NAME']='LATAM'
all_dates3 = pd.concat([all_dates, all_dates2], axis=0)
grouped_by_day_df = df[["CASE_CREATED_DATE","CASE_NBR","PROPERTY_SUPER_REGN_NAME"]]
new_df = all_dates3.merge(right=grouped_by_day_df, how='left', on=['CASE_CREATED_DATE','PROPERTY_SUPER_REGN_NAME'])

cols = ['CASE_CREATED_DATE','PROPERTY_SUPER_REGN_NAME']

# Aggregate Cases by Created Date
new_df2 = new_df.groupby(cols, as_index=False)['CASE_NBR'].count()

# Change the chart size
plt.figure(figsize=(12,6)) 

# set colors when defining the detail level
ax = sns.lineplot(data=new_df2, 
                        x='CASE_CREATED_DATE', 
                        y='CASE_NBR', 
                        hue='PROPERTY_SUPER_REGN_NAME',
                        palette={'LATAM':'blue', 'AMER':'red'}) 

# labels and title
ax.set(xlabel='Case Created Date', 
       ylabel='Cases Created', 
       title='AMER vs LATAM CopyCatcher Cases Created Daily')

# start y-axis at zero
ax.set_ylim(bottom=0)

# rotate the x axis value by 45% to make it readable 
plt.xticks(rotation=45)

plt.show()

InterfaceError: ('IM002', '[IM002] [Microsoft][ODBC Driver Manager] Data source name not found and no default driver specified (0) (SQLDriverConnect)')